# Creating Price Impact Curves

Here we create price impact curves and store them in our custom `ExternalMarket` object. At a high level, the external market is a statiscal model trained on quotes from the 1inch API. 

Every hour, we query and store quotes from swapping all permutations of tokens over various randomly distributed trade sizes. We do this using `np.geomspace` for a hard-coded min and max trade size for each token (stored in `config`, and somewhat arbitrary, but reasonable). We randomize the geomspace by scaling each point by a noise component. Let's call each set of API calls a <b>round</b>.

In this script, we pull these quotes from our Postgres database, group them by pair, and then perform a simple KNN regression to create our external market. This external market can then "predict" the price impact of trading a particular pair. These predictions are very good for most trade sizes, but become pretty noisy at the extremes (e.g. swapping 10M USDP in one go). We are working on it.

We could do simpler things like a linear or polynomial regression on our data. This works when the market is liquid enough to handle a given trade (i.e., price impact is roughly linear), but performs really poorly for large trade sizes as price impact quickly (and non-linearly) balloons to 100%. 

### Isotonic Regression

The basic intuition for fitting price impact curves is that price impact should increase monotonically with trade size. Given a set of quotes, we want to fit a regression that: (1) minimizes the error between the actual price impacts and the predicted price impacts, and (2) is monotonically increasing, (3) is bounded within [0, 1]. An Isotonic Regression conforms with all three requirements. 

An Isotonic regression minimizes the errors between training data and predictions, while enforcing (2) and (3). We implement it using scikit learn.

A KNN regression (described below) does not satisfy (2) or (3), and evidently overfits the data (meaning predictions become very noisy and unstable). A linear regression performs extremely poorly on non-linear data, over-indexing on the residuals of large trades, meaning it overestimates the price impact on smaller trades. A polynomial regression shares the problems of a KNN regression (overfitting) and of a linear regression (performs poorly on small trades).

### KNN Regression [Deprecated]

To address the non-linearity of price impact, we instead interpolate the price impact based on the k nearest trade sizes to the trade size we are trying to test. We specify `k` as `n * k_scale`. Here `n` is the number of rounds contained in the dataset, and `k_scale` is a float, like 3/2. Since in expectation each round of API calls will always hit the same points from the `np.geomspace`, this `k` ensures that we look at neighbors outside the expected points along the `np.geomspace` line.

Really, a KNN regression just allows us to fit the data more precisely than a linear regression by considering clusters of trade sizes. Otherwise, our regression would over-index on the residuals of the really large trades (e.g. swapping 100M USDP).

In [4]:
from src.sim import Scenario
from src.configs import TOKEN_DTOs, SYMBOL_TO_ADDRESS
from src.plotting import plot_quotes, plot_regression, plot_predictions

In [8]:
# TODO allow us to change the start and end time of quote fetching
scenario = Scenario("baseline", "weth")
scenario.update_market_prices(scenario.pricepaths[0])
quotes = scenario.quotes
quotes.head()

[INFO][16:41:03][src.sim.scenario]-980012: Using 731520 1Inch quotes.
[INFO][16:41:03][src.sim.scenario]-980012: Fetching sim_market from subgraph.
[INFO][16:41:06][src.utils.poolgraph]-980012: Found 20 valid cycles of length 3.


in_amount  \
src                                        dst                                                        
0x0000000000085d4780b73119b644ae5ecd22b376 0x18084fba666a33d37592fa2633fd49a74dd93a88  6.951928e+22   
                                           0x18084fba666a33d37592fa2633fd49a74dd93a88  1.000000e+21   
                                           0x18084fba666a33d37592fa2633fd49a74dd93a88  1.832981e+21   
                                           0x18084fba666a33d37592fa2633fd49a74dd93a88  3.359818e+21   
                                           0x18084fba666a33d37592fa2633fd49a74dd93a88  6.158482e+21   

                                                                                         out_amount  \
src                                        dst                                                        
0x0000000000085d4780b73119b644ae5ecd22b376 0x18084fba666a33d37592fa2633fd49a74dd93a88  1.910806e+18   
                                           0x18084fba666a33d37592fa2633fd49a74dd93a88  2.717648e+16   
                                           0x18084fba666a33d37592fa2633fd49a74dd93a88  5.037405e+16   
                                           0x18084fba666a33d37592fa2633fd49a74dd93a88  9.233241e+16   
                                           0x18084fba666a33d37592fa2633fd49a74dd93a88  1.692359e+17   

                                                                                          price  \
src                                        dst                                                    
0x0000000000085d4780b73119b644ae5ecd22b376 0x18084fba666a33d37592fa2633fd49a74dd93a88  0.000027   
                                           0x18084fba666a33d37592fa2633fd49a74dd93a88  0.000027   
                                           0x18084fba666a33d37592fa2633fd49a74dd93a88  0.000027   
                                           0x18084fba666a33d37592fa2633fd49a74dd93a88  0.000027   
                                           0x18084fba666a33d37592fa2633fd49a74dd93a88  0.000027   

                                                                                        timestamp  \
src                                        dst                                                      
0x0000000000085d4780b73119b644ae5ecd22b376 0x18084fba666a33d37592fa2633fd49a74dd93a88  1700158548   
                                           0x18084fba666a33d37592fa2633fd49a74dd93a88  1700158538   
                                           0x18084fba666a33d37592fa2633fd49a74dd93a88  1700158539   
                                           0x18084fba666a33d37592fa2633fd49a74dd93a88  1700158541   
                                           0x18084fba666a33d37592fa2633fd49a74dd93a88  1700158543   

                                                                                                     hour  \
src                                        dst                                                              
0x0000000000085d4780b73119b644ae5ecd22b376 0x18084fba666a33d37592fa2633fd49a74dd93a88 2023-11-16 18:00:00   
                                           0x18084fba666a33d37592fa2633fd49a74dd93a88 2023-11-16 18:00:00   
                                           0x18084fba666a33d37592fa2633fd49a74dd93a88 2023-11-16 18:00:00   
                                           0x18084fba666a33d37592fa2633fd49a74dd93a88 2023-11-16 18:00:00   
                                           0x18084fba666a33d37592fa2633fd49a74dd93a88 2023-11-16 18:00:00   

                                                                                       reference_price  \
src                                        dst                                                           
0x0000000000085d4780b73119b644ae5ecd22b376 0x18084fba666a33d37592fa2633fd49a74dd93a88         0.000028   
                                           0x18084fba666a33d37592fa2633fd49a74dd93a88         0.000028   
                                           0x18084fba666

## Fitting Price Impact Curves to Data

Using an illiquid pair as an example. Here we show the expected price impact of swapping WBTC into TUSD using the 1inch router.

In [9]:
def plot(
    quotes, market, i, j, scale="log", fn_regr=None, fn_pred=None, plot_prices=True
):
    in_token = market.coins[i]
    out_token = market.coins[j]

    quotes_ = quotes.loc[in_token.address, out_token.address]
    if plot_prices:
        _ = plot_quotes(quotes_, in_token.symbol, out_token.symbol)
    _ = plot_regression(quotes_, i, j, market, scale=scale, fn=fn_regr)
    _ = plot_predictions(quotes_, i, j, market, scale=scale, fn=fn_pred)

In [11]:
token1 = TOKEN_DTOs[SYMBOL_TO_ADDRESS["USDC"]]
token2 = TOKEN_DTOs[SYMBOL_TO_ADDRESS["USDP"]]
pair = (token1, token2)

market = scenario.markets[pair]

KeyError: (TokenDTO(address='0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48', name='USD Coin', symbol='USDC', decimals=6, min_trade_size=1000.0, max_trade_size=100000000.0), TokenDTO(address='0x8e870d67f660d95d5be530380d0ec0bd388289e1', name='Pax Dollar', symbol='USDP', decimals=18, min_trade_size=1000.0, max_trade_size=100000000.0))

In [ ]:
# token1 -> token2
plot(quotes, market, 0, 1)

NameError: name 'plot' is not defined

In [ ]:
# token2 -> token1
plot(quotes, market, 1, 0)

NameError: name 'plot' is not defined

## Fitting all Markets

The logic for fitting the price impact curves for all the markets is in `scripts/gen_slippage_plots.py`. The plots can be found in `figs/price_impacts`.